In [1]:
import numpy as np
from Source import RS64_Encoder
from Source import RS64_Decoder
from Source.GF64 import GFE

import keras
from keras.layers import Dense, Input,Conv1D, MaxPooling1D, Flatten, Embedding, Reshape, ReLU, Add, Concatenate, Average
from keras.models import Sequential, Model
from keras.models import load_model

Using TensorFlow backend.


In [2]:
m = 6
t_RS = 8
b_RS = 1
order_alpha=(2**m)-1
length_parity=2*t_RS
n_RS=order_alpha
k_RS=n_RS - length_parity

Z_RS=[]
for i in range(2*t_RS):    
    Z_RS.append((b_RS+i)%order_alpha)
    
g_x = RS64_Encoder.Generation_Polynomial(Z_RS, order_alpha)

codeword_size = 1

In [3]:
model = load_model('model/model_p6_sigmoid.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [4]:
total_model_SER = np.zeros(11)
total_ae_model_SER = np.zeros(11)
file_buffer = 'Result/model.csv'
model_f = open(file_buffer, 'w')

In [5]:
max_error_count = 500

In [7]:
for i in range(0, 11):
    model_SER = 0
    print('snr', i)
    total_error_count = 0
    code_count = 0
    add_model_SER=0
    
    total_ae_error_count = 0
    ae_code_count = 0
    add_ae_model_SER=0
    while(1):
        msg_RS, codeword_RS = RS64_Encoder.Encoder(order_alpha, k_RS, codeword_size, g_x)
        vector_codeword_RS = RS64_Encoder.trans_vector(codeword_RS, codeword_size, n_RS, m)
        vector_codeword_RS = np.where(vector_codeword_RS > 0, -1, 1)
        
        Eb_No = i
        recived_vector_RS = RS64_Decoder.Receive_Bit(Eb_No, vector_codeword_RS, k_RS, n_RS)
        
        model_ae_code = model.predict(recived_vector_RS.reshape(-1,6), verbose=0)
        model_RS = RS64_Decoder.Trans_symbol(model_ae_code.flatten(), m)
       

        syndrome_RS, syndrome_check = RS64_Decoder.Syndrome_Check(t_RS, 
                                                                  model_RS,
                                                                  order_alpha, Z_RS)
        
        model_estimated_code = RS64_Decoder.Decoder(syndrome_RS, syndrome_check, 
                                                    model_RS, 
                                                    t_RS, order_alpha, m)
        
        error_count, model_SER = RS64_Decoder.SER_Calculation(np.array(model_estimated_code), msg_RS,length_parity)
        ae_error_count, ae_model_SER = RS64_Decoder.SER_Calculation(np.array(model_ae_code), msg_RS,length_parity)
        total_error_count += error_count
        add_model_SER +=model_SER
        
        total_ae_error_count += ae_error_count
        add_ae_model_SER +=ae_model_SER
        
        code_count += 1
        
        if total_error_count >= max_error_count:
            break
        if code_count == 10**5:
            break
        
    total_model_SER[i] = add_model_SER / code_count
    total_ae_model_SER[i] = add_ae_model_SER/code_count
    print(code_count,total_model_SER[i])
    print(code_count,total_ae_model_SER[i])
    write_buffer = 'Eb_No, %f, ' %(i)
    model_f.write(write_buffer)
    
    write_buffer = 'SER, %.10f\n' % (total_model_SER[i])
    model_f.write(write_buffer)
    
model_f.close()

snr 0


IndexError: too many indices for array

In [ ]:
model_f.close()

In [ ]:
from scipy import special
bpsk_BER = np.zeros(11)
for i in range(0,11):
    bpsk_BER[i] = 0.5 * special.erfc((np.sqrt(10**(i/10))))

In [ ]:
import matplotlib.pyplot as plt
x = np.array(range(0, 11))

plt.semilogy(x,total_model_SER, label='model', color='b')


plt.semilogy(x,bpsk_BER, label='bpsk', color='k')


plt.xlabel('Eb/No')
plt.ylabel('SER')

plt.legend()
plt.grid(True)
plt.show()